In [1]:
from SBM_SDE import *
import obs_and_flow_classes_and_functions as old
import obs_and_flow as new
import torch
from torch import nn
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 100 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 1001
piter = 10
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
obs_error_scale = 1 / 3

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object =D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [6]:
def train_old_flow(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFLOW(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE) #For old obs_and_flow_classes_and_functions.py
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #For old obs_and_flow_classes_and_functions. #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [7]:
def train_deb_update(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    #net = SDEFLOW(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE) #For old obs_and_flow_classes_and_functions.py
    net = SDEFLOW(DEVICE, obs_model, STATE_DIM, T, DT, N, I_S_TENSOR, I_D_TENSOR, cond_inputs = 3).to(DEVICE) #For obs_and_flow.py
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    PARAMS_DICT_TENSOR = {k: torch.tensor(v).expand(BATCH_SIZE) for k, v in PARAMS_DICT.items()}
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            #C_PATH, log_prob = net() #For old obs_and_flow_classes_and_functions. #Obtain paths with solutions at times after t0.
            C_PATH, log_prob = net(BATCH_SIZE) #For obs_and_flow.py
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                #neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean() #For old obs_and_flow_classes_and_functions.py
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH, PARAMS_DICT_TENSOR) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [8]:
obs_times, obs_means, obs_error = old.csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, obs_error_scale)
obs_model = old.ObsModel(devi, obs_times, dt, obs_means[:-1, :], obs_error[:, :-1])

obs_tile_intmd = obs_model.mu[None, :, 1:, None].repeat(batch_size, state_dim_SCON, 1, 50)
#print(obs_tile_intmd)
print(obs_tile_intmd.size())
obs_tile = obs_tile_intmd.reshape(batch_size, -1, state_dim_SCON)
print(obs_tile.size())

torch.Size([2, 9, 10, 50])
torch.Size([2, 1500, 3])


In [9]:
base_dist =D.normal.Normal(loc = 0., scale = 1.)
eps = base_dist.sample([batch_size, state_dim_SCON * n, 1]).to(devi)
print(eps.size())
log_prob = base_dist.log_prob(eps).sum(1)
print(log_prob)

torch.Size([2, 1500, 1])
tensor([[-2172.9302],
        [-2091.7649]])


In [10]:
times = torch.arange(dt, t + dt, dt, device = eps.device)[(None,) * 2].repeat(batch_size, state_dim_SCON, 1).transpose(-2, -1)
print(times.size())
times_re = times.reshape(batch_size, -1, 1)
print(times_re.size())

torch.Size([2, 500, 3])
torch.Size([2, 1500, 1])


In [11]:
train_old_flow(devi, l_r, niter, piter, batch_size, old.SDEFlow, old.ObsModel, old.csv_to_obs_df, 'CON_synthetic_sol_df.csv', obs_error_scale, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/1001 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000004571.254297. Best norm loss value is: 45712.54296875.

C_PATH mean = tensor([[0.8863, 0.8348, 0.7248],
        [0.8806, 0.8348, 0.7406]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.8599,  0.5781,  0.7618],
         [ 0.8693,  0.7377,  0.6897],
         ...,
         [ 1.3962,  0.6522,  0.7266],
         [ 1.1462,  0.7405,  0.5639],
         [ 0.6915,  0.7957,  0.7501]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.7290,  0.7423,  0.8205],
         [ 0.9134,  0.3420,  0.8350],
         ...,
         [ 0.8363,  0.7548,  0.5682],
         [ 0.6518,  0.8424,  0.5077],
         [ 0.8008,  0.8265,  0.6978]]], grad_fn=<CatBackward>)



Train Diffusion:   1%|          | 10/1001 [00:04<07:16,  2.27it/s]

Moving average norm loss at 10 iterations is: 43166.308203125. Best norm loss value is: 41479.921875.

C_PATH mean = tensor([[4.6929, 0.1606, 0.4786],
        [4.4665, 0.1602, 0.4773]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.3384,  3.9361,  0.2703],
         [ 0.5741,  0.8515,  0.2652],
         ...,
         [ 2.1229,  0.2116,  0.6410],
         [ 7.4578,  0.2628,  0.3476],
         [ 1.4254,  0.2711,  0.2109]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6161,  1.5162,  0.1988],
         [ 0.4288,  3.7447,  0.2704],
         ...,
         [11.9534,  0.2841,  0.7520],
         [ 2.0646,  0.2657,  0.5994],
         [10.0231,  0.2399,  0.3122]]], grad_fn=<CatBackward>)



Train Diffusion:   2%|▏         | 20/1001 [00:09<07:23,  2.21it/s]

...


Train Diffusion:  99%|█████████▉| 990/1001 [08:02<00:04,  2.40it/s]

Moving average ELBO loss at 990 iterations is: -3531.0940185546874. Best ELBO loss value is: -4568.142578125.

C_PATH mean = tensor([[9.3921, 0.1360, 0.6174],
        [9.3928, 0.1395, 0.7032]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5495e+01, 3.2879e-02, 8.7715e-02],
         [4.5159e+01, 9.9436e-02, 1.7559e-01],
         ...,
         [2.0209e+00, 8.6255e-02, 6.2848e-01],
         [2.0171e+00, 6.2193e-02, 6.9408e-01],
         [3.4963e-01, 1.1278e-01, 5.6378e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.6240e+01, 1.1276e-01, 2.8765e-01],
         [4.5310e+01, 6.1879e-02, 4.2180e-01],
         ...,
         [4.9749e-01, 1.6906e-01, 1.1276e+00],
         [2.9393e-01, 1.8271e-01, 9.1594e-01],
         [4.0266e-01, 9.5362e-02, 8.6099e-01]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|█████████▉| 1000/1001 [08:07<00:00,  2.18it/s]

Moving average ELBO loss at 1000 iterations is: -4069.5633544921875. Best ELBO loss value is: -4568.142578125.

C_PATH mean = tensor([[9.4844, 0.1526, 0.5853],
        [9.5249, 0.1502, 0.5740]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5441e+01, 4.2252e-02, 2.7054e-01],
         [4.4569e+01, 2.5998e-02, 4.2121e-01],
         ...,
         [1.6929e+00, 1.8639e-01, 7.5679e-01],
         [1.7911e+00, 1.1675e-01, 6.0471e-01],
         [3.6986e-01, 6.6381e-02, 5.2889e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.4676e+01, 1.0955e-01, 9.3758e-02],
         [4.4760e+01, 1.6363e-01, 1.6381e-01],
         ...,
         [4.7710e-01, 6.9763e-02, 7.2625e-01],
         [2.4977e-01, 1.1861e-01, 8.2808e-01],
         [3.4075e-01, 1.3530e-01, 8.7624e-01]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 1001/1001 [08:08<00:00,  2.05it/s]


In [12]:
train_deb_update(devi, l_r, niter, piter, batch_size, new.SDEFlow, new.ObsModel, new.csv_to_obs_df, 'CON_synthetic_sol_df.csv', obs_error_scale, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/1001 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000004576.273046. Best norm loss value is: 45762.73046875.

C_PATH mean = tensor([[0.8997, 0.7692, 0.9548],
        [0.9029, 0.7680, 0.9602]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.0756,  0.6577,  0.7970],
         [ 1.4359,  0.7315,  0.7789],
         ...,
         [ 0.7723,  0.8354,  1.4406],
         [ 0.9339,  0.8026,  0.9660],
         [ 0.8354,  0.8091,  1.0106]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.9145,  0.9018,  0.7602],
         [ 1.1399,  0.8629,  0.7739],
         ...,
         [ 0.7806,  0.7353,  1.0519],
         [ 0.6222,  0.7813,  1.2944],
         [ 0.7630,  0.7526,  0.8788]]], grad_fn=<CatBackward>)



Train Diffusion:   1%|          | 10/1001 [00:05<08:28,  1.95it/s]

Moving average norm loss at 10 iterations is: 43557.0359375. Best norm loss value is: 41852.36328125.

C_PATH mean = tensor([[4.5551, 0.3658, 0.4276],
        [4.3166, 0.3504, 0.4252]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.4829,  0.6719,  0.3852],
         [ 5.2026,  0.9046,  0.3423],
         ...,
         [ 0.3255,  0.3181,  0.3288],
         [ 0.4954,  0.3538,  0.5422],
         [ 1.0524,  0.3824,  0.5214]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.9968,  1.0972,  0.3161],
         [ 2.3696,  0.3580,  0.3145],
         ...,
         [ 1.2803,  0.3341,  0.6262],
         [ 1.4185,  0.3862,  0.3258],
         [ 0.8382,  0.2678,  0.4504]]], grad_fn=<CatBackward>)



Train Diffusion:   2%|▏         | 20/1001 [00:09<07:21,  2.22it/s]

...


Train Diffusion:  99%|█████████▉| 990/1001 [08:21<00:04,  2.47it/s]

Moving average ELBO loss at 990 iterations is: -4417.878271484375. Best ELBO loss value is: -4664.37060546875.

C_PATH mean = tensor([[11.8093,  0.1396,  0.5525],
        [11.8146,  0.1323,  0.5707]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.6162e+01, 1.0006e-01, 2.3306e-01],
         [4.6001e+01, 1.4143e-01, 2.2409e-01],
         ...,
         [7.3450e-01, 1.4567e-01, 6.0641e-01],
         [4.5623e-01, 1.4242e-01, 8.0109e-01],
         [1.8818e-01, 1.3367e-01, 9.2417e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5164e+01, 1.8763e-01, 3.5456e-02],
         [4.4532e+01, 8.6396e-02, 2.1211e-01],
         ...,
         [7.7566e-01, 4.9857e-02, 6.3439e-01],
         [3.6266e-01, 4.7300e-02, 5.3458e-01],
         [3.0428e-01, 4.7697e-02, 4.2774e-01]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|█████████▉| 1000/1001 [08:25<00:00,  2.46it/s]

Moving average ELBO loss at 1000 iterations is: -4524.21279296875. Best ELBO loss value is: -4670.431640625.

C_PATH mean = tensor([[11.5986,  0.1334,  0.6711],
        [11.6405,  0.1358,  0.6592]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [45.6858,  0.1349,  0.1997],
         [45.8296,  0.0746,  0.1924],
         ...,
         [ 0.2572,  0.1860,  0.9471],
         [ 0.1251,  0.1767,  0.8561],
         [ 0.1372,  0.0922,  0.9428]],

        [[45.6603,  0.0715,  0.7147],
         [44.7451,  0.0684,  0.0728],
         [44.1997,  0.1107,  0.1826],
         ...,
         [ 1.0213,  0.0573,  0.6615],
         [ 0.5001,  0.0508,  0.7253],
         [ 0.1766,  0.1139,  0.5390]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 1001/1001 [08:25<00:00,  1.98it/s]


In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', obs_error_scale, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', obs_error_scale, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', obs_error_scale, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', obs_error_scale, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)

In [ ]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', obs_error_scale, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)